In [4]:
from lxml import html  
import json
import requests
import json,re
from dateutil import parser as dateparser
from time import sleep

def ParseReviews(asin):
    #This script has only been tested with Amazon.com
    amazon_url  = 'http://www.amazon.in/dp/'+asin
    # Add some recent user agent to prevent amazon from blocking the request 
    # Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    page = requests.get(amazon_url,headers = headers).text

    parser = html.fromstring(page)
    XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
    XPATH_REVIEW_SECTION = '//div[@id="revMHRL"]/div'
    XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
    XPATH_PRODUCT_NAME = '//h1//span[@id="productTitle"]//text()'
    XPATH_PRODUCT_PRICE  = '//span[@id="priceblock_ourprice"]/text()'

    raw_product_price = parser.xpath(XPATH_PRODUCT_PRICE)
    product_price = ''.join(raw_product_price).replace(',','')

    raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
    product_name = ''.join(raw_product_name).strip()
    total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
    reviews = parser.xpath(XPATH_REVIEW_SECTION)

    ratings_dict = {}
    reviews_list = []

    #grabing the rating  section in product page
    for ratings in total_ratings:
        extracted_rating = ratings.xpath('./td//a//text()')
        if extracted_rating:
            rating_key = extracted_rating[0] 
            raw_raing_value = extracted_rating[1]
            rating_value = raw_raing_value
            if rating_key:
                ratings_dict.update({rating_key:rating_value})

    #Parsing individual reviews
    for review in reviews:
        XPATH_RATING  ='./div//div//i//text()'
        XPATH_REVIEW_HEADER = './div//div//span[contains(@class,"text-bold")]//text()'
        XPATH_REVIEW_POSTED_DATE = './/a[contains(@href,"/profile/")]/parent::span/following-sibling::span/text()'
        XPATH_REVIEW_TEXT_1 = './/div//span[@class="MHRHead"]//text()'
        XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
        XPATH_REVIEW_COMMENTS = './/a[contains(@class,"commentStripe")]/text()'
        XPATH_AUTHOR  = './/a[contains(@href,"/profile/")]/parent::span//text()'
        XPATH_REVIEW_TEXT_3  = './/div[contains(@id,"dpReviews")]/div/text()'
        raw_review_author = review.xpath(XPATH_AUTHOR)
        raw_review_rating = review.xpath(XPATH_RATING)
        raw_review_header = review.xpath(XPATH_REVIEW_HEADER)
        raw_review_posted_date = review.xpath(XPATH_REVIEW_POSTED_DATE)
        raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_1)
        raw_review_text2 = review.xpath(XPATH_REVIEW_TEXT_2)
        raw_review_text3 = review.xpath(XPATH_REVIEW_TEXT_3)

        author = ' '.join(' '.join(raw_review_author).split()).strip('By')

        #cleaning data
        review_rating = ''.join(raw_review_rating).replace('out of 5 stars','')
        review_header = ' '.join(' '.join(raw_review_header).split())
        review_posted_date = dateparser.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
        review_text = ' '.join(' '.join(raw_review_text1).split())

        #grabbing hidden comments if present
        if raw_review_text2:
            json_loaded_review_data = json.loads(raw_review_text2[0])
            json_loaded_review_data_text = json_loaded_review_data['rest']
            cleaned_json_loaded_review_data_text = re.sub('<.*?>','',json_loaded_review_data_text)
            full_review_text = review_text+cleaned_json_loaded_review_data_text
        else:
            full_review_text = review_text
        if not raw_review_text1:
            full_review_text = ' '.join(' '.join(raw_review_text3).split())

        raw_review_comments = review.xpath(XPATH_REVIEW_COMMENTS)
        review_comments = ''.join(raw_review_comments)
        review_comments = re.sub('[A-Za-z]','',review_comments).strip()
        review_dict = {
                            'review_comment_count':review_comments,
                            'review_text':full_review_text,
                            'review_posted_date':review_posted_date,
                            'review_header':review_header,
                            'review_rating':review_rating,
                            'review_author':author

                        }
        reviews_list.append(review_dict)

    data = {
                'ratings':ratings_dict,
                'reviews':reviews_list,
                'url':amazon_url,
                'price':product_price,
                'name':product_name
            }
    return data


def ReadAsin():
    #Add your own ASINs here 
    AsinList = ['B072LPF91D','B017HW9DEW']
    extracted_data = []
    for asin in AsinList:
        print( "Downloading and processing page http://www.amazon.com/dp/"+asin)
        extracted_data.append(ParseReviews(asin))
        sleep(5)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)

if __name__ == '__main__':
    ReadAsin()

In [13]:
'''
review class of Amazon -> cr-original-review-content

## lets forget about how to get exact product link for a minute and focus on getting reviews first.
https://www.amazon.in/dp/B072LPF91D <- iphone x
'''
from bs4 import BeautifulSoup as bs
import requests
import time
headers = {'User-Agent': 'Mozilla/5.0'}

# search = 'https://www.amazon.in/dp/B072LPF91D'
search = 'https://www.amazon.in/Mi-Poco-64GB-Steel-Blue/dp/B07H4X4F5Y'

review_list = []
home_page = requests.get(search,headers=headers)
home_content = home_page.content
soup = bs(home_content, 'html.parser')
# print(soup)
for review in soup.findAll("div", {"class": "reviewText"}):
#     print(review)
    print(review.text.strip())
    review_list.append(review.text.strip())

df = pd.DataFrame(review_list,columns = ['Reviews'])
# df.to_csv('iPhone.csv',index = False)
df.to_csv('reviews.csv',index=False)
    
# for review in soup.findAll("div", {"a": "review-title-content"}):
#     print(review.text)



I want to refund box Quality bad
The product is a real deal. The battery life is amazing! I am very much satisfied with all the features of the phone.
So far its good , I will update the rating and review incase something comes up!
Lightning fast
Mine i just 2 months and the screen though claimed to be gorilla glass, had few cracks with a 1 feet fall. fortunately the display is intact and have to use it like this now since screen is not covered by warranty. I was an mi lover and third mi phone this, but not any longer
Beast under 20k phone. which can beat any Brand's phone in price bracket 30 k. You can check on YouTube all recommend this device
Very nice mobile
I have mobile


In [2]:
import requests
from bs4 import BeautifulSoup as bs

def gsearch_to_amaazon_links(search_term = "apple+iphone+x"):
    
    amazon_url_list = []
    g_search = 'https://www.google.com/search?&q=' + search_term + 'amazon'
    response = requests.get(g_search)
    
    soup = bs(response.text,'html.parser')
    results = soup.find_all('div', class_='g')

    for result in results:

        a_tag = result.find('a')

    #     print(link['href'])

        if '/dp/' in a_tag['href']:

            link = a_tag['href']
            url = link.split('=')
            url = url[1].split('&')[0]
            print(url)
            amazon_url_list.append(url)
    
    return amazon_url_list

In [3]:
# gsearch_to_amaazon_links('2 ton ka ac')
import pandas as pd

df = pd.DataFrame(gsearch_to_amaazon_links('pocofone'),columns = ['Link'])
df.to_csv('iPhone.csv',index = False)

https://www.amazon.com/Xiaomi-Pocophone-Factory-Unlocked-Smartphone/dp/B07GZ4QJTS
https://www.amazon.com/Xiaomi-Pocophone-Graphite-Unlocked-Warranty/dp/B07H41NB46
https://www.amazon.in/Mi-Poco-64GB-Steel-Blue/dp/B07H4X4F5Y
https://www.amazon.co.uk/Xiaomi-Pocophone-Dual-64GB-Blue/dp/B07GY6GVHF
https://www.amazon.co.uk/Xiaomi-Pocophone-F1-Black-64GB/dp/B07GTD2XDF


In [11]:
df = pd.read_csv('reviews.csv')
df['Reviews'][0].strip()

'I want to refund box Quality bad'

In [14]:
from textblob import TextBlob

In [33]:
tb = TextBlob(df['Reviews'][1].strip())
tb

TextBlob("The product is a real deal. The battery life is amazing! I am very much satisfied with all the features of the phone.")

In [28]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [32]:
tb.sentiment_assessments

Sentiment(polarity=0.4833333333333334, subjectivity=0.7333333333333334, assessments=[(['real'], 0.2, 0.30000000000000004, None), (['amazing', '!'], 0.7500000000000001, 0.9, None), (['very', 'much', 'satisfied'], 0.5, 1.0, None)])

In [20]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [35]:
from fuzzywuzzy import fuzz

simple_ratio=fuzz.ratio("Hello world", "Hello world!")

print(simple_ratio)

96


c:\users\ayush\desktop\virtualenvironments\fuzzy\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [36]:
pip install python-Levenshtein

  Running setup.py install for python-Levenshtein: started
    Running setup.py install for python-Levenshtein: finished with status 'error'
    Complete output from command c:\users\ayush\desktop\virtualenvironments\fuzzy\scripts\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\ayush\\AppData\\Local\\Temp\\pip-install-93kwoohf\\python-Levenshtein\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\ayush\AppData\Local\Temp\pip-record-px3hskmh\install-record.txt --single-version-externally-managed --compile --install-headers c:\users\ayush\desktop\virtualenvironments\fuzzy\include\site\python3.7\python-Levenshtein:
    running install
    running build
    running build_py
    creating build
    creating build\lib.win32-3.7
    creating build\lib.win32-3.7\Levenshtein
    copying Levenshtein\StringMatcher.py -> build\lib.win32-3.7\Levenshtein
    copying Le

Command "c:\users\ayush\desktop\virtualenvironments\fuzzy\scripts\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\ayush\\AppData\\Local\\Temp\\pip-install-93kwoohf\\python-Levenshtein\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\ayush\AppData\Local\Temp\pip-record-px3hskmh\install-record.txt --single-version-externally-managed --compile --install-headers c:\users\ayush\desktop\virtualenvironments\fuzzy\include\site\python3.7\python-Levenshtein" failed with error code 1 in C:\Users\ayush\AppData\Local\Temp\pip-install-93kwoohf\python-Levenshtein\


In [40]:
from fuzzywuzzy import fuzz

simple_ratio=fuzz.ratio("a","b")

print(simple_ratio)

0


In [42]:
from fuzzywuzzy import fuzz
 
partial=fuzz.partial_ratio("Hello world", "bye world!")
 
print(partial)

70


In [43]:
from fuzzywuzzy import process
 
choices = ["Hello world is an introductory phrase in programming", 
           "Game of throne is a movie", 
           "Albert Einstein developed the theory of relativity", 
           "Asia is the largest continent in the world"]
 
extract_all=process.extract("Hello world", choices, limit=2)
 
print(extract_all)

[('Hello world is an introductory phrase in programming', 90), ('Asia is the largest continent in the world', 86)]
